In [1]:
!pip install tensorflow numpy matplotlib

In [ ]:
Preprocess the Dataset

In [20]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os
from tensorflow.keras.preprocessing.image import img_to_array

# Directory containing handwritten text images
dataset_path = r"C:\Users\uzman\Documents\code soft\Task 5"

# Function to preprocess images
def preprocess_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (28, 28))
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=-1)
    return img

# Load the dataset
images = []
labels = []

for filename in os.listdir(dataset_path):
    if filename.endswith('.png') or filename.endswith('.jpg'):
        img = preprocess_image(os.path.join(dataset_path, filename))
        images.append(img)
        labels.append(os.path.splitext(filename)[0])

images = np.array(images)

print(f'Total images: {len(images)}')


Total images: 9


In [ ]:
Define the RNN Model

In [48]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed, Activation

# Model architecture
def build_model(vocab_size, embedding_dim=256, rnn_units=512, batch_size=64):
    model = Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        LSTM(rnn_units,
             return_sequences=True,
             stateful=False,  # Change to False unless you have a specific reason to use stateful LSTM
             recurrent_initializer='glorot_uniform'),
        Dropout(0.2),
        TimeDistributed(Dense(vocab_size)),
        Activation('softmax')
    ])
    return model

# Example usage
vocab_size = len(set(labels))
embedding_dim = 256
rnn_units = 512

model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=64)
model.summary()



Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_12 (Embedding)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_4 (TimeDistributed) │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_4 (Activation)            │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
Train the Model

In [50]:
# Example training loop
def train_model(model, dataset, epochs=10, batch_size=64):
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

    # Training loop
    for epoch in range(epochs):
        print(f'Starting epoch {epoch+1}/{epochs}')
        for batch in dataset:
            loss = model.train_on_batch(batch['input'], batch['target'])
        print(f'Epoch {epoch+1}/{epochs} loss: {loss}')

# Prepare the data for training
# Assuming that we have a function to create input and target sequences
# Here we create a dummy dataset
vocab_size = 1000  # Placeholder, replace with len(set(labels)) in actual use
embedding_dim = 256
rnn_units = 512
batch_size = 64  # Define batch_size here

# Create dummy data for input and target sequences
dummy_input = np.random.randint(vocab_size, size=(batch_size, 100))  # replace with real data
dummy_target = np.random.randint(vocab_size, size=(batch_size, 100))  # replace with real data

# Package data into the dataset
dataset = [{'input': dummy_input, 'target': dummy_target}]

# Build the model
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size)

# Train the model
train_model(model, dataset, epochs=10, batch_size=batch_size)

Starting epoch 1/10
Epoch 1/10 loss: 6.907624244689941
Starting epoch 2/10
Epoch 2/10 loss: 6.90678071975708
Starting epoch 3/10
Epoch 3/10 loss: 6.905799865722656
Starting epoch 4/10
Epoch 4/10 loss: 6.904725551605225
Starting epoch 5/10
Epoch 5/10 loss: 6.903481960296631
Starting epoch 6/10
Epoch 6/10 loss: 6.9015583992004395
Starting epoch 7/10
Epoch 7/10 loss: 6.897881507873535
Starting epoch 8/10
Epoch 8/10 loss: 6.893133640289307
Starting epoch 9/10
Epoch 9/10 loss: 6.887988567352295
Starting epoch 10/10
Epoch 10/10 loss: 6.882984161376953


In [58]:
def generate_text(model, start_string, num_generate=1000):
    input_eval = [labels.index(s) for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)

        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(labels[predicted_id])

    return (start_string + ''.join(text_generated))

# Generate text
start_string = "TOJIHKaMV1KPOBb"
generated_text = generate_text(model, start_string)
print(generated_text)


ValueError: 'T' is not in list